### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [2]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojure.java.io :as io]
            [clojure.data.csv :as csv]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

In [ ]:
(defn- csv->map
  "Convert parsed CSV vectors into maps with headers as keys, by row"
  [csv-data]
  (map zipmap ;; make the first row as headers and the following rows as values in a map structure e.g. {:tic AAPL} 
       (->> (first csv-data) ;; take the first row of the csv-data
            (map keyword) ;; make the header be the "key" in the map 
            repeat)      ;; repeat the process for all the headers
       (rest csv-data))) ;; use the rest rows as values of the map
(defn read-csv-lazy
  "Read CSV data into memory by row"
  [filename]
  (csv->map (lazy-read-csv filename)))

In [ ]:
(defn lazy-read-csv
  [csv-file]
  (let [in-file (io/reader csv-file)
        csv-seq (csv/read-csv in-file)
        lazy (fn lazy [wrapped]
               (lazy-seq
                 (if-let [s (seq wrapped)]
                   (cons (first s) (lazy (rest s)))
                   (.close in-file))))]
    (lazy csv-seq)))
(take 10 (lazy-read-csv "/Users/lyc/Downloads/data/data-CRSP.csv"))

### Import dataset

In [ ]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(def y (reset! data-set (add-aprc (read-csv-lazy "/Users/lyc/Downloads/data/data-CRSP.csv"))))

In [ ]:
(def k(swap! data-set rest))

In [13]:
(defn lazy
    [wrapped]
    (lazy-seq
        (if-let [s (seq wrapped)]
           (cons (first s) (lazy (rest s)))
           )))
(defn maintaintics 
    []
    (def ticsinfo (atom {}))
   (println "-----")
   (loop [count 0 remaining (lazy-seq (add-aprc (read-csv-lazy "/Users/lyc/Downloads/data/data-CRSP.csv"))) cur-tic nil start-date nil end-date nil num 0 reference nil]
     (if (empty? remaining)
       (do
         (if (not= cur-tic nil)
           (swap! ticsinfo assoc cur-tic {:start-date start-date :end-date end-date :pointer (atom {:num num :reference nil})}))
         nil)
       (let [first-line (first remaining)
             next-remaining (rest remaining)]
         (if (not= cur-tic (get first-line :TICKER))
           (do
             ;(println "-----")
             (if (not= cur-tic nil)
               (swap! ticsinfo assoc cur-tic {:start-date start-date :end-date end-date :pointer (atom {:num num :reference nil})}))
             (let [cur-tic (get first-line :TICKER) start-date (get first-line :date) end-date (get first-line :date) num count reference remaining]
               (recur (inc count) next-remaining cur-tic start-date end-date num (lazy reference))))
           (if (= (get (first remaining) :date) (get-date))
             (recur (inc count) next-remaining cur-tic start-date (get first-line :date) count (lazy remaining))
             (recur (inc count) next-remaining cur-tic start-date (get first-line :date) num (lazy reference)))))))
   )

#'clojure-backtesting.demo/maintaintics

In [16]:
(maintaintics)

*Interrupted*


-----
AAGIY
CBNF
AAIR
AALX
AALS
AAL
AAME
AEN
AAMN
AAPL
AARN
AASS
ABAI
AB
ABAN
ABLD
ABCC
ABEF
ABEV
ALEC
ABIG
ABI
ABKC
ABKP
AZBW
ABLA
ABLE
ABMD
ABNC

ASQ
ASO
ABNJ
ABOT
ABPC
ABUD
BUD
ACAD
ACAL
ACCM
ACCMA
ACCMB
ACCOB
RKY
TAP
ACCU
PCES
ACEL
ACES
AVCS
ACET
GCMN
ACFC
ACHP
ACIC
ACAP
UINS
ACIG
ACLE
ACLP
ACLV
UNII
ACME
ACMT
ACMTA
ACOK
ACOM
INUC
ACRA
ACRF
ACSC
ACSH
ACTC
ACTP

ACYT
ADAC
ADAR
AE

ADBE
ADCO
LDDSA
LDDS
WCOM
MCIT
MCWE
ADDC
ADDS
ADS
BKS
ADE
ADEV
ADE
ADGE
RELM
RWC
BKTI
ADGN
ADIT
ADLA
ADME
ADI
ADND
SCIO
ADRI
FAST
ADSN

ADC
ADSC
ADSO
ADVC
ADVN
ADVS
ASY
AEX

AEIC
DOSK
DOSKV
DOSK
FBAI
FDB
AELB
AELNA
AENG
ASE


interrupted: 

In [15]:
(get(deref (get (get (deref ticsinfo) "AMBC") :pointer)) :num)

651367

### Initialise portfolio （Go back here everytime you want to restart.）

In [ ]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 100000);

In [ ]:
(count (deref tics-info))

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 200 cross below the MA 50 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [ ]:
(time (do (def MA50-vec-aapl [])
          (def MA200-vec-aapl [])
          (def MA50-vec-f [])
          (def MA200-vec-f [])
          (while (not= (get-date) "2016-12-29")
            (do
    ;; write your trading strategy here
              (def tics (deref available-tics-)) ;20 ms
              (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl (get (get tics "AAPL"):reference)))
              (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl (get (get tics "AAPL") :reference)))
              (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f (get (get tics "F"):reference)))
              (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f (get (get tics "F") :reference)))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
                (if (> MA50 MA200)
                  (order "AAPL" 1 :reference (get (get tics "AAPL") :reference) :print false) 
                  (order "AAPL" 0 :remaining true :reference (get (get tics "AAPL") :reference))))
              (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
                (if (> MA50 MA200)
                  (order "F" 1 :reference (get (get tics "F") :reference) :print false) 
                  (order "F" 0 :remaining true :reference (get (get tics "F") :reference))))
              ;(update-eval-report (get-date))
              (next-date)))))
(.close wrtr)

In [ ]:
(count (deref order-record))


### Check portfolio record

In [ ]:
;; view final portfolio
(view-portfolio)

In [ ]:
;; view portfolio value and return
(view-portfolio-record)

### Generate evaluation report

In [ ]:
(eval-report)

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)